In [40]:
import plotly.express as px
import plotly.graph_objects as go
import os
import re
import numpy as np
from scipy import signal
from PIL import Image

In [34]:
def read_log(log):
    loss = open(log, 'r').read()
    out = []
    for m in re.findall(": ([\d\.]+)", loss):
        out.append(float(m))

    out = np.array(out)
    losses = np.reshape(out,(-1,8))
    return losses

In [39]:
losses = read_log("checkpoints\cel_shading_face_one_light\loss_log.txt")
losses_full = read_log("checkpoints\cel_shading\loss_log.txt")

def smooth(x):
    return signal.savgol_filter(x, 53, 3)

def plot_losess(fig, losses, prefix):
    fig.add_trace(go.Scatter(x=x, y=smooth(losses[:,4]),
                    mode='lines',
                    name=prefix + 'G_GAN'))
    fig.add_trace(go.Scatter(x=x, y=smooth(losses[:,5]),
                        mode='lines',
                        name=prefix + 'G_L1'))
    fig.add_trace(go.Scatter(x=x, y=smooth(losses[:,6]),
                        mode='lines',
                        name=prefix + 'D_real'))
    fig.add_trace(go.Scatter(x=x, y=smooth(losses[:,7]),
                        mode='lines',
                        name=prefix + 'D_fake'))

x = np.arange(len(losses[:,0]))

fig = go.Figure()

plot_losess(fig, losses, "one light ")
plot_losess(fig, losses_full, "full ")

fig.show()

In [58]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def calculate_err(path):
    files = os.listdir(path)
    sums = []
    for ff in chunks(files, 3):
        fake = np.asarray(Image.open(os.path.join(path, ff[0])))
        real = np.asarray(Image.open(os.path.join(path, ff[2])))
        sums.append(np.abs(np.sum(real-fake)))
    return sums
        
        
losses = calculate_err('./results/cel_shading_face_one_light_unet/test_latest/images')
losses.sort()
#cel_face_one_light
losses_def = calculate_err('./results/cel_shading_face_one_light/test_latest/images')
losses_def.sort()

losses_full = calculate_err('./results/cel_shading/test_latest/images')
losses_full.sort()

x = np.arange(len(losses))
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=losses,
                    mode='lines',
                    name='one light unet'))
fig.add_trace(go.Scatter(x=x, y=losses_def,
                    mode='lines',
                    name='one light'))
fig.add_trace(go.Scatter(x=x, y=losses_full,
                    mode='lines',
                    name='full'))

In [61]:
import os
import sys

base_cmd = "python3 train.py --dataroot ../../scratch/datasets/cel_face2 --name {} --model pix2pix --display_id -1 --batch_size 100"
base_test_cmd = "python3 test.py --dataroot ../../scratch/datasets/cel_face2 --name {} --model pix2pix"
#base_cmd = "python3 train.py --dataroot datasets/cel_face2 --name {} --model pix2pix --display_id -1 --batch_size 100"
#base_test_cmd = "python3 test.py --dataroot datasets/cel_face2 --name {} --model pix2pix"
base_name = "cel_face2"

experement = 4#int(sys.argv[1])

if experement == 0:
    name = base_name + "_default"
    cmd = base_cmd.format(name)
    os.system(cmd)
    os.system(base_test_cmd.format(name))
elif experement == 1:
    name = base_name + "_unet"
    args = " --netG unet_256"
    cmd = base_cmd.format(name)
    os.system(cmd + args)
    os.system(base_test_cmd.format(name) + args)
elif experement == 2:
    name = base_name + "_no_dropout"
    args = " --no_dropout"
    cmd = base_cmd.format(name)
    os.system(cmd + args)
    os.system(base_test_cmd.format(name) + args)
elif experement == 3:
    name = base_name + "_big_d"
    args = " --netD n_layers --n_layers_D 64"
    cmd = base_cmd.format(name)
    os.system(cmd + args)
    os.system(base_test_cmd.format(name) + args)
elif experement == 4:
    name = base_name + "_more_epoch"
    args = ""
    cmd = base_cmd.format(name)
    os.system(cmd + args + " --n_epochs 300 --n_epochs_decay 300")
    os.system(base_test_cmd.format(name) + args)